In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Part 1- Data Preprocessing ---------- Remove irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Part 2 - Encoding categorical data
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
#one-hot encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
OneHot_Encoder=OneHotEncoder()
geoEncoder=OneHot_Encoder.fit_transform(data[['Geography']])
geoEncoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [8]:
OneHot_Encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_Encoder=pd.DataFrame(geoEncoder.toarray(),columns=OneHot_Encoder.get_feature_names_out(['Geography']))
geo_Encoder

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
#concat the new one-hot encoded columns to the original dataframe and drop the Geography column
data=pd.concat([data.drop(['Geography'],axis=1),geo_Encoder],axis=1)


In [10]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [31]:
#sve the encoder and scaler objects
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('OneHotEncoder.pkl','wb') as f:
    pickle.dump(geo_Encoder,f)

In [ ]:
with open('label_encoder_gender.pkl','rb') as f:
    read=pickle.load(f)
    

AttributeError: 'LabelEncoder' object has no attribute 'categories_'

In [11]:
#divide dataset into independent features and dependent features
X=data.drop('Exited',axis=1)
Y=data['Exited']
#split the dataset into training and testing sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
# X_train.head()

# X_test.head()


#Scaler features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
import pickle 
with open('X_train_scaler.pkl','wb') as f:
    pickle.dump(X_train,f)
with open('X_test_scaler.pkl','wb') as f:
    pickle.dump(X_test,f)    

In [ ]:
with open('scaler_write.pkl','wb') as f:
    pickle.dump(scaler,f)

TypeError: write() argument must be str, not bytes

In [12]:
#ANN Implementation- Import libraries
import tensorflow as tf


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [14]:
model=Sequential([Dense(64,activation='relu',input_shape=(X_train.shape[1],)) ,# HL1,
                 Dense(32,activation='relu') ,#HL2,
                Dense(1,activation='sigmoid' )])#HL3

c:\Users\2401722\OneDrive - Cognizant\Documents\PythonLearn\Venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
(X_train.shape[1],)

(12,)

In [28]:
import tensorflow
op=tensorflow.keras.optimizers.Adam(learning_rate=0.1)
# loss=tensorflow.keras.losses.binary_crossentropy()

In [29]:
model.compile(optimizer=op,loss="binary_crossentropy",metrics=['accuracy'])

In [30]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
eraly_stopping=EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [31]:
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
                     callbacks=[tensorflow_callback,eraly_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8043 - loss: 0.4532 - val_accuracy: 0.8045 - val_loss: 0.4030
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8136 - loss: 0.4314 - val_accuracy: 0.8035 - val_loss: 0.4150
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8260 - loss: 0.4029 - val_accuracy: 0.8180 - val_loss: 0.4034
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8266 - loss: 0.4082 - val_accuracy: 0.8325 - val_loss: 0.3894
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8338 - loss: 0.4033 - val_accuracy: 0.8120 - val_loss: 0.4432
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8415 - loss: 0.3883 - val_accuracy: 0.8560 - val_loss: 0.3736
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8462 - loss: 0.3902 - val_accuracy: 0.8390 - val_loss: 0.4562
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8447 - loss: 0.4036 - val_accu

In [32]:
model.save('model.h5')

In [33]:
# TensorBoard
%load_ext tensorboard

In [37]:
%tensorboard --logdir logs/fit/20250905-153215

Reusing TensorBoard on port 6008 (pid 17052), started 22:31:11 ago. (Use '!kill 17052' to kill it.)